In [1]:
"""
NOTEBOOK 03: ANOTACIÓN Y VISUALIZACIÓN DE DATOS MICROVASCULARES
================================================================
Objetivo: Cargar, validar y visualizar anotaciones manuales de densidad vascular
y velocidades individuales de vasos obtuvadas de videos de microscopia intravital
sublingual.

Etapas:
1. Cargar CSV con anotaciones manuales
2. Validar calidad de datos
3. Visualizar distribuciones de conteos por categoría
4. Analizar densidades vasculares (SVD, TVD)
5. Analizar velocidades individuales
6. Estudiar tendencias temporales
7. Generar reportes

Autor: Tesis Doctoral - Análisis Automatizado de Dinámicas Microvasculares
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuración de visualización para publication-ready
sns.set_style("whitegrid")
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['legend.fontsize'] = 10

Matplotlib is building the font cache; this may take a moment.


In [ ]:
"""
TAREA 8: Guardar Datos Procesados y Reportes
==========================================
Salida del proceso de anotación y análisis:
1. CSV limpio con todos los datos procesados para pipeline de segmentación
2. Reporte de resumen en texto con estadísticas clave
3. Checkpoint de calidad de datos antes de análisis computacional

Archivos de salida:
- annotations_processed.csv: Dataset completo lista para análisis
- annotations_summary_report.txt: Reporte ejecutivo de estadísticas
"""

print("\n" + "=" * 70)
print("ETAPA 8: GUARDADO DE DATOS PROCESADOS Y REPORTES")
print("=" * 70)

# Guardar dataset procesado completo
output_path = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/annotations_processed.csv')
df_clean.to_csv(output_path, index=False)
print(f"\n✓ Dataset procesado guardado:")
print(f"  Ruta: {output_path.name}")
print(f"  Tamaño: {df_clean.shape[0]} registros × {df_clean.shape[1]} variables")

# Seleccionar velocidades no-nulas para estadísticas finales
velocity_cols = [col for col in df_clean.columns if col.startswith('velocity_')]
all_velocities = []
for col in velocity_cols:
    velocities = df_clean[col].dropna().values
    all_velocities.extend(velocities)
all_velocities = np.array(all_velocities)

# Generar reporte de resumen
summary_report = f"""
{'='*80}
REPORTE DE ANÁLISIS DE ANOTACIONES MICROVASCULARES
{'='*80}

INFORMACIÓN DEL ANÁLISIS:
- Fecha de procesamiento: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
- Notebook: 03_annotation_and_visualization.ipynb
- Objetivo: Validación y exploración de datos anotados manualmente para
   calibración de algoritmos automáticos de segmentación

{'='*80}
COHORTE DE PACIENTES
{'='*80}

Total de Registros: {len(df_clean)}
Pacientes Únicos: {df_clean['patient'].nunique()}
Pacientes: {', '.join(sorted(df_clean['patient'].unique()))}

Registros por Paciente:
{df_clean.groupby('patient').size().to_string()}

Cobertura Temporal:
- Rango de días: {df_clean['day_num'].min()} a {df_clean['day_num'].max()}
- Videos por registro: 1-3

{'='*80}
CONTEOS DE VASOS POR CATEGORÍA DE FLUJO
{'='*80}

CATEGORÍAS:
  0 = Sin flujo (estasis vascular)
  1 = Flujo intermitente (hipoperfusión intermitente)
  2 = Flujo lento (hipoperfusión pero flujo presente)
  3 = Flujo continuo (perfusión normal/adecuada)

ESTADÍSTICAS DESCRIPTIVAS (Media ± Std):
- Sin flujo (Cat 0):        {df_clean['count_cat0'].mean():6.1f} ± {df_clean['count_cat0'].std():5.1f}
- Flujo intermitente (Cat 1): {df_clean['count_cat1'].mean():6.1f} ± {df_clean['count_cat1'].std():5.1f}
- Flujo lento (Cat 2):      {df_clean['count_cat2'].mean():6.1f} ± {df_clean['count_cat2'].std():5.1f}
- Flujo continuo (Cat 3):   {df_clean['count_cat3'].mean():6.1f} ± {df_clean['count_cat3'].std():5.1f}

TOTALES DE VASOS:
- Media:  {df_clean['total_vessels'].mean():6.1f} vasos
- Std:    {df_clean['total_vessels'].std():6.1f}
- Rango:  {df_clean['total_vessels'].min():.0f} - {df_clean['total_vessels'].max():.0f} vasos

{'='*80}
DENSIDADES VASCULARES
{'='*80}

Small Vessel Density (SVD):
- Media:  {df_clean['svd'].mean():.4f}
- Std:    {df_clean['svd'].std():.4f}
- Rango:  {df_clean['svd'].min():.4f} - {df_clean['svd'].max():.4f}

Total Vessel Density (TVD):
- Media:  {df_clean['tvd'].mean():.4f}
- Std:    {df_clean['tvd'].std():.4f}
- Rango:  {df_clean['tvd'].min():.4f} - {df_clean['tvd'].max():.4f}

Correlación SVD-TVD: {df_clean[['svd', 'tvd']].corr().iloc[0, 1]:.3f}

{'='*80}
VELOCIDADES INDIVIDUALES DE VASOS
{'='*80}

Total de mediciones: {len(all_velocities)}
Promedio por registro: {len(all_velocities) / len(df_clean):.1f} mediciones

Estadísticas Generales:
- Media:    {np.mean(all_velocities):8.1f}
- Mediana:  {np.median(all_velocities):8.1f}
- Std Dev:  {np.std(all_velocities):8.1f}
- Rango:    {np.min(all_velocities):.0f} - {np.max(all_velocities):.0f}
- Q1 (25%): {np.percentile(all_velocities, 25):8.1f}
- Q3 (75%): {np.percentile(all_velocities, 75):8.1f}

Outliers (>10000): {len(all_velocities[all_velocities > 10000])} mediciones detectadas

{'='*80}
CONTROL DE CALIDAD
{'='*80}

Valores Faltantes:
- patient:      {df_clean['patient'].isna().sum()} ({100*df_clean['patient'].isna().sum()/len(df_clean):.1f}%)
- visit_code:   {df_clean['visit_code'].isna().sum()} ({100*df_clean['visit_code'].isna().sum()/len(df_clean):.1f}%)
- total_vessels: {df_clean['total_vessels'].isna().sum()} ({100*df_clean['total_vessels'].isna().sum()/len(df_clean):.1f}%)
- svd:          {df_clean['svd'].isna().sum()} ({100*df_clean['svd'].isna().sum()/len(df_clean):.1f}%)
- tvd:          {df_clean['tvd'].isna().sum()} ({100*df_clean['tvd'].isna().sum()/len(df_clean):.1f}%)

Validación Lógica:
- Consistencia categoría-total: {((df_clean['count_cat0'] + df_clean['count_cat1'] + df_clean['count_cat2'] + df_clean['count_cat3']) != df_clean['total_vessels']).sum()} inconsistencias

{'='*80}
ARCHIVOS DE SALIDA GENERADOS
{'='*80}

✓ annotations_processed.csv
  → Dataset completo con variables procesadas

✓ annotations_summary_report.txt
  → Este reporte

✓ Gráficos generados:
  - 04_vessel_category_distribution.png (distribuciones por categoría)
  - 05_vessel_density_analysis.png (análisis SVD/TVD)
  - 06_vessel_velocity_analysis.png (distribuciones de velocidad)
  - 07_temporal_trends.png (tendencias temporales)

{'='*80}
PRÓXIMOS PASOS
{'='*80}

1. Notebook 04: Segmentación automática de vasos (training)
2. Notebook 05: Análisis de videos con máscara de segmentación
3. Notebook 06: Extracción de características espacio-temporales
4. Comparación: Métricas manuales vs. automáticas para validación

{'='*80}
FIN DEL REPORTE
{'='*80}
"""

# Guardar reporte
report_path = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/annotations_summary_report.txt')
with open(report_path, 'w') as f:
    f.write(summary_report)

print(f"\n✓ Reporte de resumen guardado:")
print(f"  Ruta: {report_path.name}")

# Mostrar resumen en consola
print("\n" + summary_report)

print("\n" + "="*70)
print("✓ ETAPA 03 COMPLETADA EXITOSAMENTE")
print("="*70)
print(f"Todos los datos y gráficos han sido guardados en: src/data/")
print("Ready for Notebook 04: Vessel Segmentation Training")

## Task 7: Save Processed Data for Further Analysis

In [ ]:
"""
TAREA 7: Análisis de Tendencias Temporales
==========================================
¿Qué es?
Para pacientes con múltiples mediciones a lo largo del tiempo,
se analiza la evolución de métricas microvasculares.

¿Por qué?
- Cambios día-a-día pueden indicar:
  ✓ Respuesta a intervención terapéutica
  ✓ Deterioro progresivo de perfusión
  ✓ Recuperación o estabilización
- Base para identificar pacientes respondedores vs no-respondedores

Metodología:
- Seleccionar solo pacientes con ≥3 días de mediciones
- Inspeccionr tendencias en: total_vessels, SVD, TVD, proporción Cat3
- Gráficos línea mostrando evolución temporal
"""

print("\n" + "=" * 70)
print("ETAPA 7: ANÁLISIS DE TENDENCIAS TEMPORALES")
print("=" * 70)

# Extraer número de día del código de visita (D1, D2, D3, etc.)
if 'day_num' not in df_clean.columns:
    df_clean['day_num'] = df_clean['day'].str.extract(r'(\d+)').astype(int)

# Identificar pacientes con datos de múltiples días
patient_day_counts = df_clean.groupby('patient')['day_num'].nunique()
patients_with_trend = patient_day_counts[patient_day_counts >= 3].index.tolist()

print(f"\nPacientes con datos temporales (≥3 días): {len(patients_with_trend)}")
if patients_with_trend:
    print(f"  Pacientes: {', '.join(patients_with_trend)}")
else:
    print("  ⚠️ No hay suficientes pacientes con datos multi-día para análisis de tendencias")

if len(patients_with_trend) > 0:
    # Crear figura con 4 subplots para métricas clave
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('TENDENCIAS TEMPORALES DE MÉTRICAS MICROVASCULARES', 
                 fontsize=14, fontweight='bold')
    
    metrics = ['total_vessels', 'svd', 'tvd', 'count_cat3']
    metric_titles = ['Total de Vasos', 'SVD (Densidad Pequeños)', 'TVD (Densidad Total)', 'Vasos Flujo Continuo']
    
    for idx, (metric, title) in enumerate(zip(metrics, metric_titles)):
        ax = axes[idx // 2, idx % 2]
        
        # Graficar línea por cada paciente
        for patient in patients_with_trend:
            patient_trend = df_clean[df_clean['patient'] == patient].sort_values('day_num')
            ax.plot(patient_trend['day_num'], patient_trend[metric], 
                   marker='o', label=patient, linewidth=2.5, markersize=8)
        
        ax.set_xlabel('Día de medición', fontsize=11)
        ax.set_ylabel(title, fontsize=11)
        ax.set_title(title, fontsize=12, fontweight='bold')
        ax.legend(fontsize=10, loc='best')
        ax.grid(True, alpha=0.3)
        ax.set_xticks(sorted(df_clean['day_num'].unique()))
    
    plt.tight_layout()
    plt.savefig(
        '/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/'
        '07_temporal_trends.png',
        dpi=300, bbox_inches='tight'
    )
    print("\n✓ Gráfico de tendencias guardado")
    plt.show()
else:
    print("\nℹ️ Se requieren más pacientes con múltiples mediciones para análisis de tendencias")

## Tarea 7: Análisis de Tendencias Temporales (Multi-Día)

**¿Para qué?** Rastrear evolución de perfusión a lo largo del internamiento  
**Criterio:** Solo pacientes con ≥3 días de mediciones  
**Métricas:** Total vasos, SVD, TVD, proporción de Cat3 (flujo continuo)  

**Interpretación:**
- Mejoría: ↑ SVD/TVD, ↑ Cat3 → Respuesta terapéutica  
- Deterioro: ↓ SVD/TVD, ↑ Cat0/1 → Mal pronóstico  
- Meseta: Estabilización clínica

In [ ]:
"""
TAREA 6: Análisis de Velocidades Individuales de Vasos
=====================================================
¿Qué es?
- Cada medición incluye velocidades de hasta 20 vasos individuales (V1-V20)
- Velocidad típica en microcirculación: 100-1000 unidades (escala de pixel/frame)
- Indica rapidez del flujo sanguíneo eritrocitario

Significado clínico:
- Velocidad↓ → Hipoperfusión, riesgo de estasis
- Variabilidad↑ → Heterogeneidad de perfusión, mal pronóstico
- Outliers → Posibles errores de medición o vasos anómalos

Procesamiento:
1. Extraer todas las mediciones de V1-V20
2. Detectar outliers (> 10000 pueden ser errores)
3. Calcular estadísticas descriptivas
4. Visualizar distribuciones por paciente
"""

print("\n" + "=" * 70)
print("ETAPA 6: ANÁLISIS DE VELOCIDADES INDIVIDUALES DE VASOS")
print("=" * 70)

# Identificar columnas de velocidad (V1, V2, ..., V20)
velocity_cols = [col for col in df_clean.columns if col.startswith('velocity_')]
velocity_cols_sorted = sorted(velocity_cols, key=lambda x: int(x.split('_')[1][1:]))

# Combinar todas las mediciones de velocidad en una sola lista
all_velocities = []
velocity_sources = []  # Para rastreabilidad

for idx, row in df_clean.iterrows():
    for col in velocity_cols_sorted:
        vel = row[col]
        if pd.notna(vel):
            all_velocities.append(vel)
            velocity_sources.append((row['patient'], col))

all_velocities = np.array(all_velocities)

print(f"\nTotal de mediciones de velocidad: {len(all_velocities)}")
print(f"Promedio de mediciones por registro: {len(all_velocities) / len(df_clean):.1f}")

# Estadísticas descriptivas
print("\nESTADÍSTICAS GENERALES DE VELOCIDADES:")
print("-" * 70)
print(f"  Media:     {np.mean(all_velocities):.2f}")
print(f"  Mediana:   {np.median(all_velocities):.2f}")
print(f"  Std Dev:   {np.std(all_velocities):.2f}")
print(f"  Mín:       {np.min(all_velocities):.2f}")
print(f"  Máx:       {np.max(all_velocities):.2f}")
print(f"  Q1 (25%):  {np.percentile(all_velocities, 25):.2f}")
print(f"  Q3 (75%):  {np.percentile(all_velocities, 75):.2f}")

# Detectar y reportar outliers
outlier_threshold = 10000  # Umbral for probable errors
outliers = all_velocities[all_velocities > outlier_threshold]
print(f"\nPotenciales outliers (>{outlier_threshold}): {len(outliers)} mediciones")
if len(outliers) > 0:
    print(f"  Rango outliers: {np.min(outliers):.0f} - {np.max(outliers):.0f}")

# Estadísticas por paciente
print("\n\nVELOCIDADES POR PACIENTE:")
print("-" * 70)
for patient in sorted(df_clean['patient'].unique()):
    patient_data = df_clean[df_clean['patient'] == patient]
    velocities = []
    for col in velocity_cols_sorted:
        velocities.extend(patient_data[col].dropna().values)
    
    if velocities:
        vel_array = np.array(velocities)
        print(f"{patient}:")
        print(f"  • Mediciones: {len(vel_array)}")
        print(f"  • Media ± Std: {np.mean(vel_array):.1f} ± {np.std(vel_array):.1f}")
        print(f"  • Rango: {np.min(vel_array):.0f} - {np.max(vel_array):.0f}")

# Visualización 1: Histograma de distribución
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('DISTRIBUCIÓN DE VELOCIDADES INDIVIDUALES DE VASOS', 
             fontsize=14, fontweight='bold')

ax = axes[0]
ax.hist(all_velocities, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
ax.set_xlabel('Velocidad del vaso (unidades)', fontsize=11)
ax.set_ylabel('Frecuencia', fontsize=11)
ax.set_title(f'Histograma de Velocidades (n={len(all_velocities)})', 
             fontsize=12, fontweight='bold')
ax.axvline(np.mean(all_velocities), color='red', linestyle='--', linewidth=2, label=f'Media: {np.mean(all_velocities):.0f}')
ax.axvline(np.median(all_velocities), color='green', linestyle='--', linewidth=2, label=f'Mediana: {np.median(all_velocities):.0f}')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')

# Visualización 2: Boxplot por paciente
ax = axes[1]
velocity_by_patient = []
patient_labels = []

for patient in sorted(df_clean['patient'].unique()):
    patient_data = df_clean[df_clean['patient'] == patient]
    velocities = []
    for col in velocity_cols_sorted:
        velocities.extend(patient_data[col].dropna().values)
    if velocities:
        velocity_by_patient.append(velocities)
        patient_labels.append(patient)

bp = ax.boxplot(velocity_by_patient, labels=patient_labels, patch_artist=True)
for patch in bp['boxes']:
    patch.set_facecolor('lightcoral')
    patch.set_alpha(0.8)

ax.set_ylabel('Velocidad del vaso (unidades)', fontsize=11)
ax.set_title('Distribución de Velocidades por Paciente', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

plt.tight_layout()
plt.savefig(
    '/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/'
    '06_vessel_velocity_analysis.png',
    dpi=300, bbox_inches='tight'
)
print("\n✓ Gráfico guardado")
plt.show()

## Tarea 6: Análisis de Velocidades Individuales (V1-V20)

**¿Qué?** Velocidad de flujo sanguíneo en vasos individuales medida en unidades de pixel/frame  
**Rango típico:** 100-1000 unidades (depende de frame rate y magnificación)  
**Variabilidad:** Heterogeneidad de perfusión indicada por desv. estándar alta  
**Outliers:** Velocidades > 10000 prob. errores de medición  

**Interpretación:** 
- Velocidad↓ → Hipoperfusión, riesgo de estasis
- Variabilidad↑ → Heterogeneidad de perfusión
- Media estable → Perfusión uniforme y adecuada

In [ ]:
"""
TAREA 5: Análisis de Densidades Vasculares (SVD y TVD)
=====================================================
¿Qué es?
- SVD (Small Vessel Density): Densidad de vasos pequeños (diámetro < 20 μm)
- TVD (Total Vessel Density): Densidad total incluyendo vasos de todos los tamaños

Unidades: Típicamente mm/mm² o cm/cm² de área de imagen microcópica

Significado clínico:
- SVD↓ (reducción) → Compromiso de la microcirculación, hipoperfusión
- TVD↓ → Rarefacción vascular, mal pronóstico en críticos
- Correlaciona con indicadores de llenado capilar y perfusión tisular

Métodos de cálculo:
- Medición manual: recuento directo en video procesado
- Referencia: área de medición estandarizada
"""

print("\n" + "=" * 70)
print("ETAPA 5: ANÁLISIS DE DENSIDADES VASCULARES (SVD, TVD)")
print("=" * 70)

# Estadísticas descriptivas de densidades
print("\nESTADÍSTICAS DE DENSIDADES:")
print("-" * 70)
print(f"SVD - Small Vessel Density:")
print(f"  Media:    {df_clean['svd'].mean():.3f}")
print(f"  Std:      {df_clean['svd'].std():.3f}")
print(f"  Rango:    {df_clean['svd'].min():.3f} - {df_clean['svd'].max():.3f}")
print(f"\nTVD - Total Vessel Density:")
print(f"  Media:    {df_clean['tvd'].mean():.3f}")
print(f"  Std:      {df_clean['tvd'].std():.3f}")
print(f"  Rango:    {df_clean['tvd'].min():.3f} - {df_clean['tvd'].max():.3f}")

# Estadísticas por paciente
print("\n\nDENSIDADES POR PACIENTE:")
print("-" * 70)
density_stats = df_clean.groupby('patient')[['svd', 'tvd']].agg(['mean', 'std', 'min', 'max']).round(3)
print(density_stats.to_string())

# Calcular correlación entre SVD y TVD
correlation = df_clean[['svd', 'tvd']].corr().iloc[0, 1]
print(f"\nCorrelación SVD-TVD: {correlation:.3f} (fuerte relación lineal)")

# Visualización 1: Scatter plot SVD vs TVD
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('ANÁLISIS DE DENSIDADES VASCULARES', fontsize=14, fontweight='bold')

ax = axes[0]
colors_patients = plt.cm.Set3(np.linspace(0, 1, len(df_clean['patient'].unique())))
for idx, patient in enumerate(sorted(df_clean['patient'].unique())):
    patient_data = df_clean[df_clean['patient'] == patient]
    ax.scatter(patient_data['svd'], patient_data['tvd'], 
              label=patient, s=100, alpha=0.7, 
              color=colors_patients[idx], edgecolors='black', linewidth=1)

ax.set_xlabel('Small Vessel Density (SVD)', fontsize=11)
ax.set_ylabel('Total Vessel Density (TVD)', fontsize=11)
ax.set_title(f'SVD vs TVD por Paciente (r={correlation:.2f})', fontsize=12, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

# Visualización 2: Distribución de densidades
ax = axes[1]
positions = [1, 2]
data_densities = [df_clean['svd'].dropna(), df_clean['tvd'].dropna()]
bp = ax.boxplot(data_densities, labels=['SVD', 'TVD'], patch_artist=True, positions=positions)

colors_box = ['lightblue', 'lightcoral']
for patch, color in zip(bp['boxes'], colors_box):
    patch.set_facecolor(color)
    patch.set_alpha(0.8)

ax.set_ylabel('Densidad vascular (unidades)', fontsize=11)
ax.set_title('Distribución de Densidades', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(
    '/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/'
    '05_vessel_density_analysis.png',
    dpi=300, bbox_inches='tight'
)
print("\n✓ Gráfico guardado")
plt.show()

## Tarea 5: Análisis de Densidades Vasculares (SVD y TVD)

**Métricas:**
- **SVD (Small Vessel Density):** Densidad de vasos pequeños (diámetro < 20 μm)
- **TVD (Total Vessel Density):** Densidad total incluyendo todos los diámetros

**Unidad:** Típicamente mm/mm² de área visualizada  
**Significado:** SVD↓ = Rarefacción vascular = Mal pronóstico en pacientes críticos  
**Método:** Medición manual directa en imágenes procesadas

In [ ]:
"""
TAREA 4: Análisis de Distribución por Categoría de Flujo
========================================================
¿Qué es? Las categorías representan el tipo de flujo sanguíneo en capilares:
  - Cat 0: Sin flujo (estasis vascular)
  - Cat 1: Flujo intermitente (patrón de perfusión intermitente)
  - Cat 2: Flujo lento (hipoperfusión pero flujo presente)
  - Cat 3: Flujo continuo (perfusión normal/adecuada)

¿Por qué? La proporción de vasos en cada categoría es indicador pronóstico
de la calidad de perfusión tisular y respuesta clínica.

Cálculos:
- Proporciones (%) de vasos en cada categoría
- Distribuciones visuales por paciente
- Estadísticas descriptivas
"""

print("\n" + "=" * 70)
print("ETAPA 4: ANÁLISIS DE CONTEOS POR CATEGORÍA DE FLUJO")
print("=" * 70)

# Categorías de flujo y etiquetas descriptivas
category_labels = ['Sin flujo (0)', 'Flujo intermitente (1)', 'Flujo lento (2)', 'Flujo continuo (3)']
count_cols = ['count_cat0', 'count_cat1', 'count_cat2', 'count_cat3']

# Calcular proporciones (evitar división por cero)
for i, col in enumerate(count_cols):
    prop_col = f'prop_cat{i}'
    df_clean[prop_col] = (df_clean[col] / df_clean['total_vessels'].replace(0, np.nan)) * 100

# Estadísticas por categoría
print("\nESTADÍSTICAS POR CATEGORÍA DE FLUJO:")
print("-" * 70)
for col, label in zip(count_cols, category_labels):
    mean = df_clean[col].mean()
    std = df_clean[col].std()
    print(f"{label:30s}: {mean:6.1f} ± {std:5.1f}  (rango: {df_clean[col].min():.0f}-{df_clean[col].max():.0f})")

# Proporciones promedio
print("\nProporciones promedio de categorías:")
print("-" * 70)
for i, label in enumerate(category_labels):
    prop_col = f'prop_cat{i}'
    mean_pct = df_clean[prop_col].mean()
    print(f"{label:30s}: {mean_pct:6.1f}%")

# Visualización: Distribución por categoría
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('DISTRIBUCIÓN DE CONTEOS VASCULARES POR CATEGORÍA DE FLUJO', 
             fontsize=14, fontweight='bold', y=1.00)

for idx, (col, label) in enumerate(zip(count_cols, category_labels)):
    ax = axes[idx // 2, idx % 2]
    
    # Prepare data para boxplot por paciente
    data_for_box = [df_clean[df_clean['patient'] == p][col].dropna().values 
                    for p in sorted(df_clean['patient'].unique())]
    
    # Crear boxplot
    bp = ax.boxplot(data_for_box, 
                    labels=sorted(df_clean['patient'].unique()), 
                    patch_artist=True)
    
    # Colorear boxes
    colors = plt.cm.Set3(np.linspace(0, 1, len(data_for_box)))
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    ax.set_title(label, fontsize=12, fontweight='bold')
    ax.set_ylabel('Cantidad de vasos', fontsize=11)
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(
    '/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/'
    '04_vessel_category_distribution.png', 
    dpi=300, bbox_inches='tight'
)
print("\n✓ Gráfico guardado")
plt.show()

## Tarea 4: Distribución de Vasos por Categoría de Flujo

**Categorías (clasificación clínica):**
- **0:** Sin flujo (estasis vascular - mal pronóstico)
- **1:** Flujo intermitente (hipoperfusión pendular)
- **2:** Flujo lento (hipoperfusión con perfusión presente)
- **3:** Flujo continuo (perfusión normal/adecuada)

**Interpretación clínica:** Mayor proporción de Cat3 → mejor perfusión tisular  
**Visualización:** Boxplots por paciente para detectar variabilidad individual

In [ ]:
"""
TAREA 3: Resumen del Cohorte de Pacientes
==========================================
Genera estadísticas descriptivas por paciente incluyendo:
- Número de mediciones por paciente
- Promedios y variabilidad de métricas vasculares
- Identificación de potenciales outliers por cohorte
"""

print("\n" + "=" * 70)
print("ETAPA 3: ANÁLISIS DEL COHORTE DE PACIENTES")
print("=" * 70)

# Agrupar por paciente y calcular estadísticas
patient_info = df_clean.groupby('patient').agg({
    'record_id': 'count',  # Número de registros
    'total_vessels': ['mean', 'std', 'min', 'max'],
    'svd': ['mean', 'std', 'min', 'max'],
    'tvd': ['mean', 'std', 'min', 'max']
}).round(2)

# Renombrar columnas para legibilidad
patient_info.columns = ['n_records',
                       'vessels_mean', 'vessels_std', 'vessels_min', 'vessels_max',
                       'svd_mean', 'svd_std', 'svd_min', 'svd_max',
                       'tvd_mean', 'tvd_std', 'tvd_min', 'tvd_max']

print("\nRESUMEN ESTADÍSTICO POR PACIENTE:")
print(patient_info.to_string())

# Información adicional por paciente
print("\n\nDETALLES POR PACIENTE:")
print("-" * 70)
for patient in sorted(df_clean['patient'].unique()):
    patient_data = df_clean[df_clean['patient'] == patient]
    days = sorted(patient_data['day'].unique())
    n_days = len(days)
    n_records = len(patient_data)
    print(f"\n{patient}:")
    print(f"  • Registros: {n_records}")
    print(f"  • Días de medición: {n_days} → {', '.join(days)}")
    print(f"  • Densidad vascular (SVD): {patient_data['svd'].mean():.2f} ± {patient_data['svd'].std():.2f}")

In [ ]:
"""
TAREA 2B: Control de Calidad y Validación
==========================================
Verifica la completitud de datos y detecta valores faltantes o anómalos
que podrían comprometer el análisis posterior.

¿Qué buscamos?
- Valores NaN o ausentes en columnas clave
- Proporciones de datos válidos
- Estadísticas descriptivas para detección de outliers
"""

print("\n" + "-" * 70)
print("VALIDACIÓN DE CALIDAD DE DATOS")
print("-" * 70)

# Detectar y reportar valores faltantes en columnas críticas
print("\nVAlores faltantes en columnas clave:")
key_cols = ['patient', 'visit_code', 'total_vessels', 'svd', 'tvd']
for col in key_cols:
    missing = df_clean[col].isna().sum()
    pct = 100 * missing / len(df_clean)
    status = "⚠️" if pct > 5 else "✓"
    print(f"  {status} {col:20s}: {missing:3d} faltantes ({pct:5.1f}%)")

# Estadísticas descriptivas de métricas vasculares
print("\nESTADÍSTICAS DE MÉTRICAS VASCULARES:")
print("-" * 70)

summary_cols = ['count_cat0', 'count_cat1', 'count_cat2', 'count_cat3', 'total_vessels', 'svd', 'tvd']
summary_stats = df_clean[summary_cols].describe().round(2)

print(summary_stats.to_string())

# Validaciones lógicas
print("\n\nVALIDACIONES LÓGICAS:")
print("-" * 70)

# Verificar que suma de categorías = total de vasos
df_clean['category_sum'] = df_clean['count_cat0'] + df_clean['count_cat1'] + df_clean['count_cat2'] + df_clean['count_cat3']
mismatch = (df_clean['category_sum'] != df_clean['total_vessels']).sum()
print(f"  • Inconsistencias categoría-total: {mismatch} de {len(df_clean)} registros")
if mismatch > 0:
    print("    (La suma de categorías no coincide con el total de vasos)")

# Validar rangos razonables  
print(f"  • Total de vasos: rango [{df_clean['total_vessels'].min():.0f}, {df_clean['total_vessels'].max():.0f}]")
print(f"  • SVD: rango [{df_clean['svd'].min():.2f}, {df_clean['svd'].max():.2f}]")
print(f"  • TVD: rango [{df_clean['tvd'].min():.2f}, {df_clean['tvd'].max():.2f}]")

## Tarea 2-3: Limpieza, Validación y Exploración del Cohorte

**¿Qué?** 
- Renombrar columnas con estándares consistentes
- Eliminar datos vacíos/duplicados
- Extraer información temporal (día, video)
- Detectar inconsistencias y valores faltantes

**¿Por qué?** 
- Datos limpios = análisis confiable
- Identificar problemas antes de procesamiento
- Base sólida para pipeline automático

**¿Cómo?** 
- Mapeo de columnas
- Extracción regex de componentes
- Validación lógica (suma categorías = total)

In [ ]:
"""
TAREA 2: Limpieza y reestructuración de datos
==============================================
Procesa el CSV cargado para:
1. Eliminar filas vacías (continuaciones de datos del paciente anterior)
2. Renombrar columnas con nombres estandarizados en inglés
3. Extraer componentes de fecha/hora (día, video)
4. Crear identificadores únicos por registro
5. Preparar estructura para análisis subsecuentes

MAPEO DE COLUMNAS:
- Paciente → patient (identificador de paciente)
- Tiempo (Día/Video) → visit_code (ej: D2-V1)
- 0, 1, 2, 3 → count_cat0, count_cat1, count_cat2, count_cat3 (conteos por categoría)
- TOT → total_vessels (total de vasos contados)
- Small Vessel density → svd (densidad de vasos pequeños)
- total Vessel density → tvd (densidad total de vasos)
- V1-V20 → velocity_V1 a velocity_V20 (velocidades individuales de vasos)
"""

print("\n" + "=" * 70)
print("ETAPA 2: LIMPIEZA Y REESTRUCTURACIÓN DE DATOS")
print("=" * 70)

# Eliminar filas donde Paciente es NaN (son continuaciones)
df_clean = df_raw.dropna(subset=['Paciente'])
print(f"\nFilas después de eliminar vacías: {len(df_clean)} (removidas: {len(df_raw) - len(df_clean)})")

# Crear mapeo de nombres de columnas
column_mapping = {
    'Paciente': 'patient',
    'Tiempo (Día/Video)': 'visit_code',
    '0': 'count_cat0',
    '1': 'count_cat1',
    '2': 'count_cat2',
    '3': 'count_cat3',
    'TOT': 'total_vessels',
    'Small Vessel density': 'svd',
    'total Vessel density': 'tvd'
}

# Identificar columnas de velocidad individual (V1, V2, ..., V20)
vessel_cols = [col for col in df_clean.columns if col.startswith('V') and col[1:].isdigit()]
vessel_velocity_mapping = {col: f'velocity_{col}' for col in vessel_cols}

# Combinar mapeos y renombrar columnas
all_mappings = {**column_mapping, **vessel_velocity_mapping}
df_clean = df_clean.rename(columns=all_mappings)

# Extraer componentes de visita (día y número de video)
# Asume formato: "D1-V1" (Día 1, Video 1)
df_clean[['day', 'video']] = df_clean['visit_code'].str.extract(r'(D\d+)-(V\d+)')

# Crear identificador único por registro
df_clean['record_id'] = df_clean['patient'] + '_' + df_clean['visit_code']

# Conversión de tipos de datos apropiados
numeric_cols = ['count_cat0', 'count_cat1', 'count_cat2', 'count_cat3', 'total_vessels', 'svd', 'tvd']
for col in numeric_cols:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Conversión de velocidades a numéricas
for col in [f'velocity_{v}' for v in vessel_cols]:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

print(f"  ✓ Formato del dataset: {df_clean.shape}")
print(f"  ✓ Nuevos nombres de columnas aplicados")

print("\nMuestra de datos reestructurados:")
display_cols = ['record_id', 'patient', 'visit_code', 'day', 'video', 'total_vessels', 'svd', 'tvd']
print(df_clean[display_cols].head(10).to_string())

print(f"\nPacientes únicos: {df_clean['patient'].nunique()} → {sorted(df_clean['patient'].unique())}")
print(f"Registros por paciente: {df_clean.groupby('patient').size().to_dict()}")

In [ ]:
"""
TAREA 1: Cargar y parsear datos de anotación en CSV
=====================================================
Carga el archivo Excel exportado con las anotaciones manuales de:
- Conteos de vasos por categoría de flujo (0: sin flujo, 1: flujo intermitente,
  2: flujo lento, 3: flujo continuo)
- Densidad de vasos pequeños (SVD: Small Vessel Density)
- Densidad total de vasos (TVD: Total Vessel Density)  
- Velocidades individuales de hasta 20 vasos por medición
"""

# Define ruta absoluta al archivo CSV
csv_path = Path("/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/Grilla trauma SL - Lattanzio (1) - Sheet1.csv")

# Cargar datos desde CSV
print("=" * 70)
print("ETAPA 1: CARGA DE DATOS DE ANOTACIÓN")
print("=" * 70)
print(f"\nCargando archivo: {csv_path.name}")

df_raw = pd.read_csv(csv_path)

# Información inicial de los datos
print(f"  ✓ Forma del dataset: {df_raw.shape} (filas, columnas)")
print(f"  ✓ Tamaño en memoria: {df_raw.memory_usage(deep=True).sum() / 1024:.2f} KB")

# Mostrar primeras filas
print("\nPrimeras 10 filas del dataset cargado:")
print(df_raw.head(10).to_string())

## Tarea 1: Carga de Datos de Anotación

**¿Qué?** Cargar archivo Excel (guardado como CSV) con mediciones manuales de microcirculación  
**¿Por qué?** Obtener datos de referencia para validar algoritmos automáticos  
**¿Cómo?** Pandas `read_csv()` + inspección de estructura

## Tarea 8: Guardado y Reportes

**Salidas:**
- ✓ `annotations_processed.csv`: Dataset completo procesado
- ✓ `annotations_summary_report.txt`: Reporte ejecutivo
- ✓ Gráficos publication-ready en PNG

**Propósito:** Checkpoint de validación antes de pipeline automático  
**Siguiente:** Notebook 04 - Entrenamiento de segmentador de vasos

# 03 - Anotación y visualización

## Objetivo
Visualizar segmentaciones de vasos capilares.
Editar y crear máscaras de vasos si es necesario.

## Formato
Se asume que las máscaras están en `data/annotations/` como imágenes binarias


In [ ]:
from PIL import Image
import numpy as np

frame = Image.open("../data/processed/video1/frame_0000.png")
mask = Image.open("../data/annotations/video1/frame_0000_mask.png")

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(frame, cmap='gray')
plt.title("Frame original")

plt.subplot(1, 2, 2)
plt.imshow(mask, cmap='gray')
plt.title("Máscara de vasos")
plt.show()
